# Kodlabb - Algoritmen *gradient descent*

## Förkunskaper
- Matematik 3
- Grundkunskap i Python

## Introduktion
En dator lyder instruktioner till punkt och pricka. Vad händer om en dator ombeds att ta egna beslut och att träna sig på att ta bättre och bättre beslut? Det är just det som maskininlärning (ML) och artificiell intelligens (AI) handlar om.

Inom ML programmers ofta datorer till att försöka minimera mängden fel de gör och ofta används algoritmen *gradient descent*  (GD) för just detta.

[![](https://img.youtube.com/vi/jAu1ZsTCA64/maxresdefault.jpg)](https://www.youtube.com/watch?v=jAu1ZsTCA64)
*YouTube: Datorer besegrade Dota 2 proffs i 1v1 matcher.*

### Studiestund 1

Innan vi kommer igång behöver vi utvidga våra kunskaper från Matte 3 och lära oss om *funktioner av flera variabler*, *partiell derivata* och *gradienter*.

####  Funktioner av flera variabler, partiell derivata och gradienter

En *definitionsmängd* eller en *domän* är inom matematiken mängden av alla möjliga argument eller invärden för en funktion. Funktioner av flera variabler såsom $g$ och $h$ nedan har flerdimensionella definitionsmängder.

\begin{array}{rl rl}
f(x)     &=x^2         & f(2)     &=4  \\
g(x,y)   &=x^2+y^2     & f(2,3)   &=13 \\
h(x,y,z) &=x^2+y^2+z^2 & f(2,3,4) &=29 \\
\end{array}

Ibland är det bra att veta hur funktionsvärdet förändras om vi förflyttar oss i definitionsmängden. För varje riktning vi kan röra oss inom definitionsmängden kan vi beräkna en *partiell derivata*. En *gradient* är en vektor vars komponenter består av de partiella derivatorna. Titta nu på följande film och besvara därefter några kontrollfrågor.

[![](https://img.youtube.com/vi/GkB4vW16QHI/maxresdefault.jpg)](https://www.youtube.com/watch?v=GkB4vW16QHI)
*YouTube: Du får lära dig om [partiell derivata](https://sv.wikipedia.org/wiki/Partiell_derivata) som beteckas $\frac{\partial f}{\partial x}$ och $\frac{\partial f}{\partial y}$ och om [gradienter](https://sv.wikipedia.org/wiki/Gradient) som betecknas $\nabla f$.*

### Fråga 1
I bilden ovan ser vi en funktion $z=f(x, y)$ representerad som en yta. Föreställ dig att du är en myra på den ytan.
- a) Beskriv vart du skulle gå om du följde gradienten och stod i en backe.
- b) Beskriv vart du skulle gå om du följde gradienten och stod på plan mark.
- c) Beskriv vart du skulle gå om du följde gradienten och stod på en kulles topp.
- d) Beskriv vad det engelska ordet *descent* betyder och gissa hur algoritmen *gradient descent* fungerar.

### Svar
SKRIV DITT SVAR HÄR

## Initialisering
Vi kommer använda oss av pythonbiblioteket `numpy` och lite stödkod för just denna kodlabb. Stödkoden använder sig av pythonbiblioteket `plotly` för att visa interaktiva grafer.

In [1]:
import numpy as np
from simulation import Sim

sim = Sim()

## Kodkomplettering 1
För att öva på *gradient descent* (GD) behöver vi ha en funktion.

Definiera en pythonfunktion `f` så att den beter sig precis som den matematiska funktionen $f(x, y)$ nedan. Använd dig av `numpy` bibliotekets funktioner `np.sin`, `np.cos` och `np.exp` tillsammans med Pythons inbyggda operator `**`.

$$f(x, y) = \sin(\,({\frac{x}{2}-2})^2 + (\frac{y}{2}-2)^2\,) \cdot \cos(\,x-y+e^{-y}\,)$$

In [2]:
def f(x, y):
    """En funktion att undersöka som returnerar ett värde för varje input x och y."""
    ### BEGIN SOLUTION
    return np.sin((x/2-2)**2 + (y/2-2)**2) * np.cos(x-y + np.exp(-y))
    ### END SOLUTION
    
sim.set_f(f)

## Kodkomplettering 2
*Gradient descent* bygger på att vi kan promenera antiparallellt (åt andra hållet) till gradienten, och gradienten byggs upp av de partiella derivatorna. Vi kan beräkna den partiella derivatan ungefärligt med ett litet värde på $h$ på följande sätt.

$$\frac{\partial f}{\partial x} \approx \frac{f(x+h, y) - f(x-h, y)}{2h}$$

$$\frac{\partial f}{\partial y} \approx \frac{f(x, y+h) - f(x, y-h)}{2h}$$

Det är ditt uppdrag att komplettera funktionerna nedan som numeriskt beräknar de partiella derivatorna för funktionen `f`.

In [3]:
h = 0.001
def df_dx(x, y):
    """Returns the value of the partial derivative of x for a given """
    ### BEGIN SOLUTION
    return (f(x+h,y) - f(x-h,y)) / (2*h)
    ### END SOLUTION
def df_dy(x, y):
    ### BEGIN SOLUTION
    return (f(x,y+h) - f(x,y-h)) / (2*h)
    ### END SOLUTION
    
sim.set_grad_f(df_dx, df_dy)

## Kodkomplettering 3
TODO: Förklara GD-algoritmen och variablerna gamma och precision, samt tipsa om `np.linalg.norm`

In [4]:
sim.gamma = 0.2
sim.precision = 0.001
def gradient_descent_step(x, y, gamma):
    ### BEGIN SOLUTION
    new_x = x - gamma * df_dx(x, y)
    new_y = y - gamma * df_dy(x, y)
    step_size = np.linalg.norm((new_x - x, new_y - y))
    ### END SOLUTION
    return new_x, new_y, step_size
    
sim.set_gds(gradient_descent_step)

## Testkör
TODO: Förklara starting pointsen, och ställ lite frågor..

In [5]:
sim.set_starting_points(
    (0.1, 2.75),
    (2.0, 2.0),
    (2.6, 2.6),
    (2.4, 0.2),
)

sim.run()